In [1]:
# Native
import re
import sys
import time

# Third party
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs

# Custom
sys.path.append("..")
from Classes.DB import DB
from linkedInCred import (linkedIn_email, linkedIn_password)
from scripts.companyObjects import (daugherty, slalom, _1904labs, worldWideTechnology)

In [2]:
web_data_db = DB()
web_data_db.connect()
web_data_db.createTableCompanyProfiles()
web_data_db.createTableCompanyPosts()

You are connected to -  ['PostgreSQL 13.0, compiled by Visual C++ build 1914, 64-bit'] 



In [3]:
companies = [slalom, _1904labs, worldWideTechnology]

In [4]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
browser.driver.set_window_size(1600, 900) # Has to be wide enough to prevent messenger from covering filter buttons

In [5]:
def logOnToLinkedIn():
    browser.visit(companies[0].linkedin)
    button = browser.links.find_by_partial_href("https://www.linkedin.com/login?")
    button.click()
    browser.fill('session_key', linkedIn_email)
    browser.fill('session_password', linkedIn_password)
    button = browser.find_by_value('Sign in')
    button.click()

In [6]:
def scrapeProfileMetaData(company):
    url = company.linkedin
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')

    followers = re.search(r'[\d,]+ followers', html, re.DOTALL).group()
    followers = re.sub("[^\d]", "", followers) # returns only digits
    followers = int(followers)

    employees_on_linkedin = re.search(r'See all(.*)employees on LinkedIn', html, re.DOTALL).group()
    employees_on_linkedin = re.sub("[^\d]", "", employees_on_linkedin) # returns only digits
    employees_on_linkedin = int(employees_on_linkedin)

    print(f"{company.name} has {followers} followers and {employees_on_linkedin} employees on LinkedIn.")

    web_data_db.intoCompanyProfiles(company.name, followers, employees_on_linkedin, url)
    web_data_db.select()

In [7]:
def sortProfilePostsByMostRecent():
    button = browser.find_by_css("div[class='sort-dropdown mt2 ember-view']")
    button.click()
    button = browser.find_by_text('Recent')
    button.click()

In [8]:
def scrollDownUntilAllPostsAreLoaded():
    number_posts_before_scroll = len(browser.find_by_css("div[class='occludable-update ember-view']"))

    while number_posts_before_scroll > 1:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        number_posts_after_scroll = len(browser.find_by_css("div[class='occludable-update ember-view']"))
        
        if (number_posts_before_scroll == number_posts_after_scroll):
            timer = time.time()
            ten_seconds_elapsed = timer + 10
            
            while time.time() < ten_seconds_elapsed:
                number_posts_before_scroll = len(browser.find_by_css("div[class='occludable-update ember-view']"))
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                number_posts_after_scroll = len(browser.find_by_css("div[class='occludable-update ember-view']"))
                
            if (number_posts_before_scroll == number_posts_after_scroll):
                print("finished")
                break
                
    print(f"Number of posts {number_posts_before_scroll}")
        

In [9]:
def scrapeCompanyPosts(id):
    posts = browser.find_by_css("div[class='occludable-update ember-view']")

    for post in posts:
        post = post.text
          
        try:
            date = re.search(r'^(\w+ •\n|\w+ ago\n)', post, re.MULTILINE).group()
        except Exception as e:
            print(f"no like date: {e}")
            date = ""
            
        try:
            like_count = re.search(r'^[\d]+$', post, re.MULTILINE).group()
            like_count = int(like_count)
        except Exception as e:
            print(f"no like count: {e}")
            like_count = 0

        try:
            # Content always follows the time the post was published and precedes the like count.
            contentRegex = re.compile(f"( •\n| ago\n).*?^({like_count})$", re.MULTILINE|re.DOTALL)
            content = re.search(contentRegex, post).group()
            content = re.sub(r"( •\n| ago\n)", "", content, re.MULTILINE|re.DOTALL) # Gets rid of the leading timestamp
            content = content[:-len(str(like_count))] # Gets rid of the trailing like count
        except Exception as e:
            print(f"no content: {e}")
            content = ""
            
        try:
            comment_area = re.search(r'^[\d]+ comment(s)?$', post, re.MULTILINE).group()
            comment_count = re.sub(r"[^\d]", "", comment_area) # returns only digits
            comment_count = int(comment_count)
        except Exception as e:
            print(f"no comment count: {e}")
            comment_count = 0
            
        print(f"\n\nlikes: {like_count}")
        print(f"comments: {comment_count}")
        print(f"content: {content}")
        web_data_db.intoCompanyPosts(content, like_count, comment_count, date, id + 1)
        
    web_data_db.select()
    

In [10]:
logOnToLinkedIn()

for id in range(len(companies)):
    scrapeProfileMetaData(companies[id])
    browser.visit(companies[id].linkedin)
    sortProfilePostsByMostRecent()
    scrollDownUntilAllPostsAreLoaded()
    scrapeCompanyPosts(id)

Slalom has 116221 followers and 7511 employees on LinkedIn.
([1, 'Slalom', 116221, 7511, 'https://www.linkedin.com/company/slalom-consulting/'],)
()
finished
Number of posts 49


likes: 28
comments: 1
content: In case you missed it, watch Slalom’s exclusive event with award-winning actor, producer, and activist America Ferrera. Thrilled to wrap #reInvent with this official event focused on building equity and community—which starts with inclusive teams empowered to create solutions that represent the diversity of our communities:
…see more
Watch on-demand
go.slalom.com • 1 min read



NameError: name 'likes' is not defined